# 1. Importando as bibliotecas

In [120]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# 2. Carregando dataset

In [121]:
df = pd.read_csv('ordens_case_dnc_.csv')

# 3. Visualizando dataset

In [122]:
df.head()

,id,price,order_date,due_date,opt_date,delivery_date,cidade_origem_id,regiao_origem_id,cidade_destino_id,regiao_destino_id,peso
0,0,107.78,2022-02-25 20:46:25+00:00,2022-03-02 00:00:00,2022-02-28 20:35:20,2022-03-02 18:03:33,2,1,25,1,23.8
1,1,7.47,2022-02-23 20:34:59+00:00,2022-02-24 00:00:00,2022-02-24 09:56:23,2022-02-24 15:52:40,2,1,16,4,536.0
2,2,5.98,2022-02-23 20:35:19+00:00,2022-02-24 00:00:00,2022-02-23 20:29:38,2022-02-24 22:55:38,2,1,88,1,202.0
3,3,5.98,2022-02-10 14:34:37+00:00,2022-02-17 00:00:00,2022-02-17 07:47:49,2022-02-17 14:23:12,2,1,21,1,441.0
4,4,5.98,2022-02-23 20:35:05+00:00,2022-02-24 00:00:00,2022-02-23 20:28:40,2022-02-24 12:08:00,2,1,25,1,316.0


In [123]:
df.dtypes

id                     int64
price                float64
order_date            object
due_date              object
opt_date              object
delivery_date         object
cidade_origem_id       int64
regiao_origem_id       int64
cidade_destino_id      int64
regiao_destino_id      int64
peso                 float64
dtype: object

Observa-se que as colunas order_date, due_date, opt_date, delivery_date deveriam estar no formato de datetime mas estão como objeto, logo, deverão ser tratados.

In [124]:
df.isnull().sum()

id                   0
price                0
order_date           0
due_date             0
opt_date             0
delivery_date        0
cidade_origem_id     0
regiao_origem_id     0
cidade_destino_id    0
regiao_destino_id    0
peso                 0
dtype: int64

In [125]:
# Tratando data
cols = ['order_date', 'due_date', 'opt_date', 'delivery_date']
for col in cols:
  df[col] = pd.to_datetime(df[col])

In [126]:
df.dtypes

id                                 int64
price                            float64
order_date           datetime64[ns, UTC]
due_date                  datetime64[ns]
opt_date                  datetime64[ns]
delivery_date             datetime64[ns]
cidade_origem_id                   int64
regiao_origem_id                   int64
cidade_destino_id                  int64
regiao_destino_id                  int64
peso                             float64
dtype: object

# 4. Análise Exploratória de Dados

In [127]:
df['order_date'] = df['order_date'].dt.tz_localize(None)

In [128]:
df.dtypes

id                            int64
price                       float64
order_date           datetime64[ns]
due_date             datetime64[ns]
opt_date             datetime64[ns]
delivery_date        datetime64[ns]
cidade_origem_id              int64
regiao_origem_id              int64
cidade_destino_id             int64
regiao_destino_id             int64
peso                        float64
dtype: object

In [129]:
cols = ['order_date', 'due_date', 'opt_date', 'delivery_date']
for col in cols:
  df[col]= pd.to_datetime(df[col]).dt.date

In [130]:
df.head()

,id,price,order_date,due_date,opt_date,delivery_date,cidade_origem_id,regiao_origem_id,cidade_destino_id,regiao_destino_id,peso
0,0,107.78,2022-02-25,2022-03-02,2022-02-28,2022-03-02,2,1,25,1,23.8
1,1,7.47,2022-02-23,2022-02-24,2022-02-24,2022-02-24,2,1,16,4,536.0
2,2,5.98,2022-02-23,2022-02-24,2022-02-23,2022-02-24,2,1,88,1,202.0
3,3,5.98,2022-02-10,2022-02-17,2022-02-17,2022-02-17,2,1,21,1,441.0
4,4,5.98,2022-02-23,2022-02-24,2022-02-23,2022-02-24,2,1,25,1,316.0


In [131]:
# Tratando data
cols = ['order_date', 'due_date', 'opt_date', 'delivery_date']
for col in cols:
  df[col] = pd.to_datetime(df[col])

In [132]:
df.dtypes

id                            int64
price                       float64
order_date           datetime64[ns]
due_date             datetime64[ns]
opt_date             datetime64[ns]
delivery_date        datetime64[ns]
cidade_origem_id              int64
regiao_origem_id              int64
cidade_destino_id             int64
regiao_destino_id             int64
peso                        float64
dtype: object

In [133]:
# Criando uma coluna de dias de atraso com a diferença entre a data de entrega e a data de entrega esperada
df['atraso'] = df['delivery_date'] - df['due_date']

In [134]:
# Criando uma coluna para mostrar quantos dias depois que foi feito o pedido, ele saiu para entrega
df['Lag_order_shipping'] = (df['opt_date'] - df['order_date'])

In [135]:
df.head()

,id,price,order_date,due_date,opt_date,delivery_date,cidade_origem_id,regiao_origem_id,cidade_destino_id,regiao_destino_id,peso,atraso,Lag_order_shipping
0,0,107.78,2022-02-25,2022-03-02,2022-02-28,2022-03-02,2,1,25,1,23.8,0 days,3 days
1,1,7.47,2022-02-23,2022-02-24,2022-02-24,2022-02-24,2,1,16,4,536.0,0 days,1 days
2,2,5.98,2022-02-23,2022-02-24,2022-02-23,2022-02-24,2,1,88,1,202.0,0 days,0 days
3,3,5.98,2022-02-10,2022-02-17,2022-02-17,2022-02-17,2,1,21,1,441.0,0 days,7 days
4,4,5.98,2022-02-23,2022-02-24,2022-02-23,2022-02-24,2,1,25,1,316.0,0 days,0 days


In [136]:
df['atraso'] = (df['atraso']/np.timedelta64(1, 'D')).astype(int)

In [137]:
df['Lag_order_shipping'] = (df['Lag_order_shipping']/np.timedelta64(1, 'D')).astype(int)

In [138]:
df.head()

,id,price,order_date,due_date,opt_date,delivery_date,cidade_origem_id,regiao_origem_id,cidade_destino_id,regiao_destino_id,peso,atraso,Lag_order_shipping
0,0,107.78,2022-02-25,2022-03-02,2022-02-28,2022-03-02,2,1,25,1,23.8,0,3
1,1,7.47,2022-02-23,2022-02-24,2022-02-24,2022-02-24,2,1,16,4,536.0,0,1
2,2,5.98,2022-02-23,2022-02-24,2022-02-23,2022-02-24,2,1,88,1,202.0,0,0
3,3,5.98,2022-02-10,2022-02-17,2022-02-17,2022-02-17,2,1,21,1,441.0,0,7
4,4,5.98,2022-02-23,2022-02-24,2022-02-23,2022-02-24,2,1,25,1,316.0,0,0


In [139]:
df[df['atraso'] < 0] = 0

In [202]:
df1 = df[['atraso','cidade_origem_id','cidade_destino_id']] \
.pivot_table(index='cidade_origem_id',
             columns='cidade_destino_id',
             values='atraso',
             aggfunc='mean') \
.applymap(lambda x: x if x>0 else 0)

# Análise de Atrasos por cidades de destino e origem

In [223]:
l = {}
for i in list(df1.index):
    l[f'Origem id: {i}'] = [np.mean(df1.iloc[i])]
pd.DataFrame(l,index = ['Average Delay'])

,Origem id: 0,Origem id: 1,Origem id: 2,Origem id: 3,Origem id: 4,Origem id: 5
Average Delay,0.0,0.172484,1.138897,0.150287,0.558797,0.867915


In [242]:
pd.set_option('display.max_rows',150)
l = {}
for i in list(df1.columns):
    l[f'Destino id: {i}'] = [np.mean(df1[i])]
pd.DataFrame(l,index = ['Average Delay'])\
.transpose()\
.sort_values(by='Average Delay',ascending=False)

,Average Delay
Destino id: 23,3.166667
Destino id: 111,2.666667
Destino id: 59,2.500000
Destino id: 63,2.444444
Destino id: 40,2.166667
Destino id: 87,2.000000
Destino id: 10,1.944444
Destino id: 48,1.714286
Destino id: 41,1.666667
Destino id: 34,1.650000


In [314]:
l = {}
for i in list(df1.columns):
    l[f'Destino id: {i}'] = list(pd.DataFrame(df1[df1[i] == 0][i]).index)
import json
jason_vorhees = []
for i in l.values():
    jason_vorhees.append(json.dumps(i))
pd.DataFrame(jason_vorhees,index = l.keys(),columns=['Origens Sem Atraso'])\
.sort_values(by='Origens Sem Atraso', ascending = False)

,Origens Sem Atraso
Destino id: 24,[0]
Destino id: 23,[0]
Destino id: 15,[0]
Destino id: 19,"[0, 4]"
Destino id: 10,"[0, 3]"
Destino id: 6,"[0, 3, 5]"
Destino id: 17,"[0, 2, 3, 5]"
Destino id: 92,"[0, 2, 3, 4, 5]"
Destino id: 28,"[0, 2, 3, 4, 5]"
Destino id: 2,"[0, 1]"


In [294]:
l = {}
for i in list(df1.index):
    s = df1.iloc[i] == 0 
    l[f'Origem id: {i}'] = list(s[s].index)
lista = []
for i in l.values():
    lista.append(len(i))
pd.DataFrame(lista,index = l.keys(),columns=['Nº de Destinos Sem Atraso'])\
.sort_values(by='Nº de Destinos Sem Atraso', ascending = False)

,Nº de Destinos Sem Atraso
Origem id: 0,106
Origem id: 1,97
Origem id: 3,94
Origem id: 4,76
Origem id: 5,71
Origem id: 2,68


In [253]:
l = {}
for i in list(df1.index):
    l[f'Origem id: {i}'] = list(pd.DataFrame(df1.iloc[i]).sort_values(by=i,ascending=False).index)
origins = pd.DataFrame(l)
origins.index = [f'{i}º Pior' for i in origins.index+1]
origins

,Origem id: 0,Origem id: 1,Origem id: 2,Origem id: 3,Origem id: 4,Origem id: 5
1º Pior,0,28,111,59,40,41
2º Pior,86,92,63,24,105,23
3º Pior,84,10,55,13,23,58
4º Pior,83,17,87,23,79,48
5º Pior,81,6,59,19,87,42
6º Pior,80,23,54,46,24,108
7º Pior,79,15,93,15,10,103
8º Pior,78,19,86,16,90,34
9º Pior,77,24,98,56,39,72
10º Pior,76,80,1,2,17,77


In [254]:
l = {}
for i in list(df1.columns):
    l[f'Destino id: {i}'] = list(pd.DataFrame(df1[i]).sort_values(by=i,ascending=False).index)
pd.DataFrame(l,index = ['1º Pior','2º Pior','3º Pior','4º Pior','5º Pior','6º Pior'])\
.transpose()

,1º Pior,2º Pior,3º Pior,4º Pior,5º Pior,6º Pior
Destino id: 0,0,1,2,3,4,5
Destino id: 1,2,4,5,3,0,1
Destino id: 2,2,5,3,4,0,1
Destino id: 3,0,1,2,3,4,5
Destino id: 6,1,2,4,0,3,5
Destino id: 7,0,1,2,3,4,5
Destino id: 8,4,0,1,2,3,5
Destino id: 9,5,0,1,2,3,4
Destino id: 10,1,2,4,5,0,3
Destino id: 11,0,1,2,3,4,5
